In [ ]:
import json
import requests
import re
import os
from pathlib import Path

class MeetingHelper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation"
    
    def start(self):
        print("🎉 欢迎使用会议分析助手！")
        print("="*50)
        
        while True:
            print("\n📝 请选择输入方式：")
            print("1. 直接粘贴会议内容")
            print("2. 上传文件 (支持 .txt, .docx)")
            print("3. 退出")
            
            choice = input("\n请选择 (1-3): ").strip()
            
            if choice == "1":
                meeting_text = self.get_text_input()
                if meeting_text:
                    self.process_meeting(meeting_text)
            elif choice == "2":
                meeting_text = self.get_file_input()
                if meeting_text:
                    self.process_meeting(meeting_text)
            elif choice == "3":
                print("👋 再见！")
                break
            else:
                print("❌ 请选择 1-3")
    

    
    def process_meeting(self, meeting_text):
        print("\n🔍 正在分析会议内容...")
        result = self.analyze(meeting_text)
        
        if result["ok"]:
            print("\n🎯 分析完成！")
            save_choice = input("\n💾 是否保存分析结果到文件？(y/n): ").lower().strip()
            if save_choice in ['y', 'yes', '是', '好']:
                self.save_result(result["result"])
        else:
            print(f"❌ 分析失败: {result['error']}")
    
    def save_result(self, result):
        try:
            filename = f"meeting_analysis_{self.get_timestamp()}.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            print(f"✅ 结果已保存到: {filename}")
        except Exception as e:
            print(f"❌ 保存失败: {str(e)}")
    
    def get_timestamp(self):
        from datetime import datetime
        return datetime.now().strftime("%Y%m%d_%H%M%S")
    
    def detect_language(self, text):
        chinese_chars = len(re.findall(r'[\u4e00-\u9fff]', text))
        english_chars = len(re.findall(r'[a-zA-Z]', text))
        
        if chinese_chars > english_chars:
            return "中文"
        elif english_chars > chinese_chars * 2:
            return "英文"
        else:
            return "混合"
    
    def analyze(self, meeting_text, auto_translate=False):
        language = self.detect_language(meeting_text)
        print(f"🌐 检测语言: {language}")
        
        if language == "英文":
            print("🔄 分析英文会议...")
            english_result = self.analyze_in_english(meeting_text)
            
            if english_result["ok"]:
                if not auto_translate:
                    print("\n" + "="*50)
                    print("📋 English Meeting Analysis")
                    print("="*50)
                    self.print_result_english(english_result["result"])
                    
                    print("\n" + "="*50)
                    translate_choice = input("🤔 需要翻译成中文吗？(y/n): ").lower().strip()
                    print("="*50)
                    
                    if translate_choice in ['y', 'yes', '是', '好']:
                        print("🔄 翻译中...")
                        chinese_result = self.translate_result_to_chinese(english_result["result"])
                        if chinese_result["ok"]:
                            print("✅ 翻译完成！")
                            print("\n" + "="*50)
                            print("📋 中文会议纪要")
                            print("="*50)
                            self.print_result_chinese(chinese_result["result"])
                            return chinese_result
                        else:
                            print("❌ 翻译失败，保留英文版本")
                            return english_result
                    else:
                        return english_result
                else:
                    chinese_result = self.translate_result_to_chinese(english_result["result"])
                    return chinese_result if chinese_result["ok"] else english_result
            else:
                return english_result
        else:
            print("🔄 分析中文会议...")
            chinese_result = self.analyze_in_chinese(meeting_text)
            
            if chinese_result["ok"]:
                print("\n" + "="*50)
                print("📋 中文会议纪要")
                print("="*50)
                self.print_result_chinese(chinese_result["result"])
            
            return chinese_result
    
    def analyze_in_english(self, meeting_text):
        if len(meeting_text) > 8000:
            print(f"⚠️ 会议内容较长({len(meeting_text)}字符)，将分析前8000字符")
            meeting_text = meeting_text[:8000]
        
        prompt = f"""
You are a professional meeting minutes assistant. Please extract key information from the following meeting content and return it in JSON format.

Meeting Content:
{meeting_text}

Please generate JSON in the following format:
{{
  "meeting_summary": "Brief summary of meeting content, 2-3 sentences",
  "key_topics": [
    {{
      "topic": "Topic name",
      "description": "Detailed description of the topic",
      "importance": "high/medium/low",
      "related_tasks": [
        {{
          "task": "Task name under this topic",
          "assignee": "Responsible person",
          "deadline": "Deadline",
          "priority": "high/medium/low",
          "status": "pending/in-progress/completed"
        }}
      ]
    }}
  ],
  "decisions": [
    {{
      "decision": "Decision content",
      "decision_maker": "Decision maker",
      "impact": "Impact"
    }}
  ],
  "next_steps": ["Next steps"],
  "unresolved_issues": ["Unresolved issues"]
}}
Requirements:
- Each topic can contain multiple subtasks.
- If information is unclear, fill in "unclear".
- Return must be strict JSON format without explanations.
"""
        
        try:
            response = self.call_ai(prompt)
            if response:
                data = self.parse_json(response)
                if data:
                    return {"ok": True, "result": self.fix_data_english(data)}
                else:
                    return self.simple_way_english(meeting_text)
            else:
                return {"ok": False, "error": "AI调用失败"}
        except Exception as e:
            return {"ok": False, "error": f"分析出错: {str(e)}"}
    
    def analyze_in_chinese(self, meeting_text):
        # 如果文本太长，先截取前面部分
        if len(meeting_text) > 8000:
            print(f"⚠️ 会议内容较长({len(meeting_text)}字符)，将分析前8000字符")
            meeting_text = meeting_text[:8000]
            
        prompt = f"""
你是一位专业的会议纪要助手，请根据以下会议内容，提取关键信息并以JSON格式返回。
会议内容：
{meeting_text}
请按照以下格式生成JSON：
{{
  "meeting_summary": "简要总结会议内容，2-3句话",
  "key_topics": [
    {{
      "topic": "议题名称",
      "description": "该议题的详细描述",
      "importance": "high/medium/low",
      "related_tasks": [
        {{
          "task": "该议题下的子任务名称",
          "assignee": "负责人",
          "deadline": "截止时间",
          "priority": "high/medium/low",
          "status": "待处理/进行中/已完成"
        }}
      ]
    }}
  ],
  "decisions": [
    {{
      "decision": "决策内容",
      "decision_maker": "决策人",
      "impact": "影响"
    }}
  ],
  "next_steps": ["下一步计划"],
  "unresolved_issues": ["未解决问题"]
}}
要求：
- 每个议题都可以包含多个子任务。
- 子任务字段中，如果信息不明确，请填写"未明确"。
- 返回结果必须是严格的JSON格式，不要包含解释说明或多余文字。
"""
        
        try:
            response = self.call_ai(prompt)
            if response:
                data = self.parse_json(response)
                if data:
                    return {"ok": True, "result": self.fix_data_chinese(data)}
                else:
                    return self.simple_way_chinese(meeting_text)
            else:
                return {"ok": False, "error": "AI调用失败"}
        except Exception as e:
            return {"ok": False, "error": f"分析出错: {str(e)}"}
    
    def call_ai(self, prompt):
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        
        data = {
            "model": "qwen-turbo",
            "input": {"messages": [{"role": "user", "content": prompt}]},
            "parameters": {"temperature": 0.3, "max_tokens": 2000}
        }
        
        try:
            print("🔄 正在调用AI...")
            resp = requests.post(self.api_url, headers=headers, json=data)
            print(f"📡 API响应状态: {resp.status_code}")
            
            if resp.status_code == 200:
                result = resp.json()
                print(f"🔍 API返回结构: {list(result.keys())}")
                
                # 调试：打印完整响应结构
                if 'output' in result:
                    print(f"📋 output结构: {list(result['output'].keys())}")
                    if 'choices' in result['output']:
                        return result['output']['choices'][0]['message']['content']
                    elif 'text' in result['output']:
                        return result['output']['text']
                    else:
                        print(f"⚠️ 未找到预期字段，完整响应: {result}")
                        return None
                else:
                    print(f"⚠️ 没有output字段，完整响应: {result}")
                    return None
            else:
                print(f"❌ API错误: {resp.status_code}")
                print(f"错误内容: {resp.text}")
                return None
        except Exception as e:
            print(f"❌ 请求异常: {str(e)}")
            return None
    
    def parse_json(self, text):
        try:
            clean_text = text.strip()
            if clean_text.startswith('```json'):
                clean_text = clean_text.split('```json')[1]
            if clean_text.endswith('```'):
                clean_text = clean_text.rsplit('```', 1)[0]
            return json.loads(clean_text)
        except:
            return None
    
    def translate_result_to_chinese(self, english_result):
        translate_prompt = f"""
请将以下英文会议分析结果翻译成中文，保持JSON结构不变：

{json.dumps(english_result, indent=2)}

要求：
- 保持JSON格式完全一致
- 只翻译文本内容，不改变字段名
- 翻译要自然流畅
- 直接返回翻译后的JSON，不要解释
"""
        
        try:
            response = self.call_ai(translate_prompt)
            if response:
                translated_data = self.parse_json(response)
                if translated_data:
                    return {"ok": True, "result": translated_data}
            return {"ok": False, "error": "翻译失败"}
        except:
            return {"ok": False, "error": "翻译出错"}
    
    def fix_data_english(self, data):
        basic_fields = ["meeting_summary", "key_topics", "decisions", "next_steps", "unresolved_issues"]
        for field in basic_fields:
            if field not in data:
                data[field] = [] if field != "meeting_summary" else ""
        
        for i, topic in enumerate(data.get("key_topics", [])):
            topic["id"] = f"topic_{i+1}"
            if "related_tasks" not in topic:
                topic["related_tasks"] = []
            for j, task in enumerate(topic["related_tasks"]):
                task["id"] = f"task_{i+1}_{j+1}"
                task.setdefault("status", "pending")
                task.setdefault("assignee", "unassigned")
                task.setdefault("deadline", "unclear")
                task.setdefault("priority", "medium")
        
        for decision in data.get("decisions", []):
            decision.setdefault("decision_maker", "unclear")
            decision.setdefault("impact", "unclear")
        
        return data
    
    def fix_data_chinese(self, data):
        basic_fields = ["meeting_summary", "key_topics", "decisions", "next_steps", "unresolved_issues"]
        for field in basic_fields:
            if field not in data:
                data[field] = [] if field != "meeting_summary" else ""
        
        for i, topic in enumerate(data.get("key_topics", [])):
            topic["id"] = f"议题_{i+1}"
            if "related_tasks" not in topic:
                topic["related_tasks"] = []
            for j, task in enumerate(topic["related_tasks"]):
                task["id"] = f"任务_{i+1}_{j+1}"
                task.setdefault("status", "待处理")
                task.setdefault("assignee", "未明确")
                task.setdefault("deadline", "未明确")
                task.setdefault("priority", "medium")
        
        for decision in data.get("decisions", []):
            decision.setdefault("decision_maker", "未明确")
            decision.setdefault("impact", "未明确")
        
        return data
    
    def simple_way_english(self, meeting_text):
        try:
            topic_answer = self.call_ai(f"What are the main topics?\n{meeting_text[:800]}")
            decision_answer = self.call_ai(f"What decisions were made?\n{meeting_text[:800]}")
            
            return {
                "ok": True,
                "result": {
                    "meeting_summary": "Meeting analysis completed",
                    "key_topics": [{
                        "id": "topic_1",
                        "topic": "Main Topics", 
                        "description": topic_answer or "Unable to extract",
                        "importance": "medium",
                        "related_tasks": []
                    }],
                    "decisions": [{
                        "decision": decision_answer or "No clear decisions", 
                        "decision_maker": "unclear", 
                        "impact": "unclear"
                    }],
                    "next_steps": [],
                    "unresolved_issues": []
                }
            }
        except:
            return {"ok": False, "error": "Simple extraction failed"}
    
    def simple_way_chinese(self, meeting_text):
        try:
            topic_answer = self.call_ai(f"主要讨论了什么？\n{meeting_text[:800]}")
            decision_answer = self.call_ai(f"做了什么决定？\n{meeting_text[:800]}")
            
            return {
                "ok": True,
                "result": {
                    "meeting_summary": "会议分析完成",
                    "key_topics": [{
                        "id": "议题_1",
                        "topic": "主要议题", 
                        "description": topic_answer or "无法提取",
                        "importance": "medium",
                        "related_tasks": []
                    }],
                    "decisions": [{
                        "decision": decision_answer or "无明确决策", 
                        "decision_maker": "未明确", 
                        "impact": "未明确"
                    }],
                    "next_steps": [],
                    "unresolved_issues": []
                }
            }
        except:
            return {"ok": False, "error": "简单提取失败"}
    
    def print_result_english(self, result):
        print(f"📝 Meeting Summary: {result.get('meeting_summary', 'N/A')}")
        
        print("\n🎯 Key Topics:")
        for topic in result.get('key_topics', []):
            print(f"  • {topic.get('topic', 'Unknown')} ({topic.get('importance', 'medium')} priority)")
            if topic.get('related_tasks'):
                for task in topic['related_tasks']:
                    print(f"    - {task.get('task', 'Unknown task')} → {task.get('assignee', 'Unassigned')}")
        
        print("\n✅ Decisions:")
        for decision in result.get('decisions', []):
            print(f"  • {decision.get('decision', 'Unknown decision')}")
        
        if result.get('next_steps'):
            print("\n⏭️ Next Steps:")
            for step in result['next_steps']:
                print(f"  • {step}")
        
        if result.get('unresolved_issues'):
            print("\n❓ Unresolved Issues:")
            for issue in result['unresolved_issues']:
                print(f"  • {issue}")
    
    def print_result_chinese(self, result):
        print(f"📝 会议总结：{result.get('meeting_summary', '无')}")
        
        print("\n🎯 关键议题：")
        for topic in result.get('key_topics', []):
            print(f"  • {topic.get('topic', '未知议题')} (重要性：{topic.get('importance', 'medium')})")
            if topic.get('related_tasks'):
                for task in topic['related_tasks']:
                    print(f"    - {task.get('task', '未知任务')} → {task.get('assignee', '未分配')}")
        
        print("\n✅ 决策事项：")
        for decision in result.get('decisions', []):
            print(f"  • {decision.get('decision', '未知决策')}")
        
        if result.get('next_steps'):
            print("\n⏭️ 下一步计划：")
            for step in result['next_steps']:
                print(f"  • {step}")
        
        if result.get('unresolved_issues'):
            print("\n❓ 未解决问题：")
            for issue in result['unresolved_issues']:
                print(f"  • {issue}")

def main():
    print("🚀 会议分析助手启动")
    
    api_key = input("请输入千问API密钥: ").strip()
    if not api_key:
        print("❌ API密钥不能为空")
        return
    
    helper = MeetingHelper(api_key)
    helper.start()

if __name__ == "__main__":
    main()

🚀 会议分析助手启动
🎉 欢迎使用会议分析助手！

📝 请选择输入方式：
1. 直接粘贴会议内容
2. 上传文件 (支持 .txt, .docx)
3. 退出

📋 请粘贴会议内容 (粘贴完成后按回车):
✅ 已获取 39785 字符的会议内容

🔍 正在分析会议内容...
🌐 检测语言: 英文
🔄 分析英文会议...
⚠️ 会议内容较长(39785字符)，将分析前8000字符
🔄 正在调用AI...
📡 API响应状态: 200
🔍 API返回结构: ['output', 'usage', 'request_id']
📋 output结构: ['finish_reason', 'text']

📋 English Meeting Analysis
📝 Meeting Summary: The meeting discussed Hugging Face's collaboration with Intel to showcase efficient AI model building using Intel's CPUs and AI accelerators. The focus was on demonstrating how to optimize workloads with Optimum Intel and Gaudi 3, emphasizing scalability and performance improvements.

🎯 Key Topics:
  • Hugging Cast Overview (high priority)
    - Prepare upcoming episodes → Unclear
    - Engage with live audience during demos → Hosts
  • Optimization with Optimum Intel (high priority)
    - Develop tutorials for Optimum Intel → Documentation Team
    - Test integration with popular models → Core Maintainers
  • Intel Gaudi 3 Accelerator (high pr

In [7]:
import json
import requests
import re
# os and pathlib are not needed
# from datetime import datetime # Not needed

class MeetingAnalyzerV2:
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://dashscope.aliyuncs.com/api/v1/services/aigc/text-generation/generation"
        # Increased max_tokens as summaries can be long, especially combined ones
        self.default_parameters = {"temperature": 0.3, "max_tokens": 3500} 

    def detect_language(self, text):
        # Simple language detection based on character counts
        chinese_chars = len(re.findall(r'[\u4e00-\u9fff]', text))
        english_chars = len(re.findall(r'[a-zA-Z]', text))
        
        if chinese_chars > english_chars:
            return "中文"
        elif english_chars > chinese_chars and english_chars > 20:
            return "英文"
        elif chinese_chars > 0 :
            return "中文"
        else:
            return "英文"

    def call_ai(self, prompt, parameters=None):
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        
        current_parameters = self.default_parameters.copy()
        if parameters:
            current_parameters.update(parameters)
            
        data = {
            "model": "qwen-turbo", # or qwen-long for very long contexts if needed/available
            "input": {"messages": [{"role": "user", "content": prompt}]},
            "parameters": current_parameters
        }
        
        try:
            # print(f"🔄 Calling AI with prompt (first 100 chars): {prompt[:100]}...")
            resp = requests.post(self.api_url, headers=headers, json=data, timeout=120) # Increased timeout
            # print(f"📡 API Status: {resp.status_code}")
            
            if resp.status_code == 200:
                result = resp.json()
                if 'output' in result:
                    if 'choices' in result['output'] and result['output']['choices']:
                        return result['output']['choices'][0]['message']['content']
                    elif 'text' in result['output']:
                        return result['output']['text']
                print(f"⚠️ Unexpected API response structure: {result}")
                return None
            else:
                print(f"❌ API Error: {resp.status_code} - {resp.text}")
                return f"API_ERROR: {resp.status_code} - {resp.text}" # Return error string
        except requests.exceptions.RequestException as e:
            print(f"❌ Request Exception: {str(e)}")
            return f"REQUEST_EXCEPTION: {str(e)}"
        except Exception as e:
            print(f"❌ AI Call Exception: {str(e)}")
            return f"AI_CALL_EXCEPTION: {str(e)}"
        return None


    def _split_text_into_chunks(self, text: str, max_words: int = 3000) -> list[str]:
        """Splits text into chunks of approximately max_words."""
        words = text.split() # Simple split by whitespace
        chunks = []
        for i in range(0, len(words), max_words):
            chunk_words = words[i:i + max_words]
            chunks.append(" ".join(chunk_words))
        # print(f"Split text into {len(chunks)} chunks.")
        return chunks

    def _summarize_chunk_in_language(self, chunk_text: str, is_chinese_prompt: bool) -> str:
        """Summarizes a single chunk of text using Markdown format."""
        if is_chinese_prompt:
            prompt = f"""
请你扮演一个专业的会议记录员。
针对以下文本片段，请提取核心议题和关键内容。
严格按照以下Markdown格式输出，不要添加任何额外的解释、引言或结尾：

**主题：** [此处填写识别出的主题]
**具体内容：** [此处填写该主题下的详细内容摘要]

文本片段：
---
{chunk_text}
---
请确保输出清晰、简洁，并严格遵守上述格式。
"""
        else: # English prompt
            prompt = f"""
You are a professional meeting summarizer.
For the following text segment, extract the core topic and key content.
Strictly use the following Markdown format for your output. Do not add any extra explanations, introductions, or conclusions:

**Topic:** [Insert identified topic here]
**Content:** [Insert detailed content summary for this topic here]

Text Segment:
---
{chunk_text}
---
Ensure your output is clear, concise, and strictly adheres to the format above.
"""
        # print(f"Summarizing chunk (lang: {'CH' if is_chinese_prompt else 'EN'})...")
        summary = self.call_ai(prompt)
        if summary and not (summary.startswith("API_ERROR:") or summary.startswith("REQUEST_EXCEPTION:") or summary.startswith("AI_CALL_EXCEPTION:")):
            return summary.strip()
        else:
            # print(f"Failed to summarize chunk or AI returned error: {summary}")
            error_msg_key = "主题：分块摘要失败" if is_chinese_prompt else "**Topic:** Chunk Summary Failed"
            error_msg_val = f"未能处理此文本块。错误：{summary if summary else '未知错误'}" if is_chinese_prompt else f"Could not process this text block. Error: {summary if summary else 'Unknown error'}"
            return f"{error_msg_key}\n**具体内容：** {error_msg_val}"


    def _consolidate_summaries_in_language(self, combined_summaries: str, is_chinese_prompt: bool) -> str:
        """Consolidates and deduplicates combined chunk summaries."""
        if is_chinese_prompt:
            prompt = f"""
你是一位专业的编辑，擅长整合和提炼信息。
以下是多个文本片段的初步概括，每个概括都包含“主题”和“具体内容”。
你的任务是：
1. 仔细阅读所有概括。
2. 识别并合并重复或高度相似的主题。
3. 对每个独特或合并后的主题，生成一个最终的、精炼的总结。
4. 最终输出必须严格遵循以下Markdown格式，每个主题占一组：

### 主题句1
具体内容1

### 主题句2
具体内容2
...

请确保最终输出流畅自然，逻辑清晰，并且只包含符合上述格式的Markdown内容。不要添加任何其他前缀、后缀、引言或解释性文字。

待整合的概括内容如下：
---
{combined_summaries}
---
请开始整合和提炼。
"""
        else: # English prompt
            prompt = f"""
You are an expert editor skilled in synthesizing and refining information.
Below are preliminary summaries from multiple text segments, each containing a "Topic" and "Content".
Your tasks are to:
1. Carefully read all the summaries.
2. Identify and merge duplicate or highly similar topics.
3. For each unique or merged topic, generate a final, concise summary.
4. The final output must strictly follow this Markdown format, with each topic in its own group:

### Topic Sentence 1
Detailed content for topic 1

### Topic Sentence 2
Detailed content for topic 2
...

Ensure the final output is fluent, logical, and contains ONLY Markdown content adhering to the format above. Do not add any prefixes, suffixes, introductions, or explanatory text.

Summaries to consolidate:
---
{combined_summaries}
---
Please begin consolidation and refinement.
"""
        # print(f"Consolidating summaries (lang: {'CH' if is_chinese_prompt else 'EN'})...")
        # Use higher max_tokens for consolidation
        final_result = self.call_ai(prompt, parameters={"max_tokens": 4000}) # qwen-turbo max is 8k tokens for context
        if final_result and not (final_result.startswith("API_ERROR:") or final_result.startswith("REQUEST_EXCEPTION:") or final_result.startswith("AI_CALL_EXCEPTION:")):
            return final_result.strip()
        else:
            # print(f"Failed to consolidate summaries or AI returned error: {final_result}")
            return f"### 整合失败\n无法完成最终摘要整合。原始合并摘要：\n{combined_summaries}\n错误：{final_result if final_result else '未知错误'}" \
                if is_chinese_prompt \
                else f"### Consolidation Failed\nCould not complete final summary consolidation. Original combined summaries:\n{combined_summaries}\nError: {final_result if final_result else 'Unknown error'}"


    def analyze(self, meeting_text: str) -> str:
        """
        Analyzes meeting text by splitting, summarizing chunks, and consolidating.
        Returns a Markdown string.
        """
        if not meeting_text or not meeting_text.strip():
            return "### 错误\n输入的会议内容为空。"

        language = self.detect_language(meeting_text)
        is_chinese = (language == "中文")
        # print(f"🌐 Detected language: {language}")

        # 1. Split text into chunks
        # Word count can be tricky for CJK languages. 3000 words might be ~6000-9000 characters.
        # qwen-turbo has a context window of 8k tokens. Let's use a character limit for splitting
        # to be safer, or stick to word count and hope the model handles tokenization well.
        # For words, 3000 words is a good number.
        chunks = self._split_text_into_chunks(meeting_text, max_words=2800) # Slightly less than 3000 for safety margin

        if not chunks:
            return "### 错误\n无法将文本分割成块。" if is_chinese else "### Error\nCould not split text into chunks."

        # 2. Summarize each chunk
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            # print(f"Processing chunk {i+1}/{len(chunks)}")
            summary = self._summarize_chunk_in_language(chunk, is_chinese)
            chunk_summaries.append(summary)
        
        combined_chunk_summaries = "\n\n---\n\n".join(chunk_summaries) # Separator for AI to distinguish

        # 3. Consolidate and deduplicate summaries
        final_markdown_summary = self._consolidate_summaries_in_language(combined_chunk_summaries, is_chinese)
        
        return final_markdown_summary


def analyze_meeting_text_v2(meeting_content: str, api_key: str) -> str:
    """
    Main function to analyze meeting text (V2: Markdown multi-stage).
    
    Args:
        meeting_content: The raw text of the meeting.
        api_key: Your Dashscope API key.
        
    Returns:
        A Markdown string of the analysis result, or an error message.
    """
    if not api_key:
        return "### 错误\nAPI密钥是必需的。"
    if not meeting_content or not meeting_content.strip():
        return "### 错误\n会议内容不能为空。"

    analyzer = MeetingAnalyzerV2(api_key)
    analysis_result_markdown = analyzer.analyze(meeting_content)
    
    return analysis_result_markdown

# Example Usage (main part for testing):
if __name__ == "__main__":
    # IMPORTANT: Set your API key here or via an environment variable
    try:
        API_KEY = open('qwen.key').read().strip() 
        if not API_KEY or "your_actual_api_key_here" in API_KEY: # Basic check
            raise FileNotFoundError 
    except FileNotFoundError:
        API_KEY = "sk-your_actual_api_key_here" # Placeholder
        print("🚨 API key file 'qwen.key' not found or empty. Please create it with your Dashscope API key.")

    if "your_actual_api_key_here" in API_KEY:
        print("🚨 Please replace 'sk-your_actual_api_key_here' with your actual Dashscope API key to run the example.")
    else:
        print("🚀 Meeting Analysis Tool (V2 - Markdown Output)")
        print("="*50)

        # Test with a sample English text file
        try:
            # Use a longer text file for meaningful chunking
            # Create a dummy long_english_text.txt if you don't have one
            # For example, repeat the content of 'huggingcast-s2e6 text-en.txt' multiple times
            with open('huggingcast-s2e6 text-en.txt', 'r', encoding='utf-8') as f_en:
                english_meeting_text_short = f_en.read()

            print("\n--- Analyzing English Meeting Text (expecting multiple chunks) ---")
            result_markdown_en = analyze_meeting_text_v2(english_meeting_text_short, API_KEY)
            print("\nAnalysis Result (Markdown String - English):")
            print(result_markdown_en)

            # You can add a Chinese text example similarly if you have one
            # chinese_meeting_text = """
            # [一段较长的中文会议记录，最好超过3000个“词”的概念，可能需要几千到一万个汉字]
            # 第一次项目启动会讨论了关于新产品“智能助手”的开发计划。
            # 张三：我认为我们首先需要明确产品的核心功能。是主打语音交互还是文本辅助？
            # 李四：我同意张三的看法。市场调研显示，用户对高效语音交互的需求很高。建议初期集中资源攻克语音识别和自然语言理解的准确性。
            # 王五：关于时间节点，我们能否在三个月内推出第一个beta版本？
            # 赵六：三个月比较紧张，考虑到技术难点和团队磨合，我建议四个月，确保质量。
            # ... (更多内容，确保文本足够长) ...
            # 第二部分讨论了市场推广策略。
            # 陈七：早期用户获取方面，我们考虑和一些科技KOL合作。
            # 周八：社交媒体宣传也很重要，特别是针对年轻用户群体。
            # ... (更多内容) ...
            # 最终决定，产品核心功能侧重语音交互，开发周期定为四个月，市场推广初期以KOL合作为主。
            # 下一步行动：李四负责整理详细的技术需求文档，下周一前完成。王五和赵六共同制定详细的项目排期。陈七调研合适的KOL资源。
            # """
            # print("\n--- Analyzing Chinese Meeting Text ---")
            # result_markdown_ch = analyze_meeting_text_v2(chinese_meeting_text * 5, API_KEY) # Multiply to make it long
            # print("\nAnalysis Result (Markdown String - Chinese):")
            # print(result_markdown_ch)

        except FileNotFoundError:
            print("\n⚠️ English test file 'huggingcast-s2e6 text-en.txt' not found. Skipping English example.")
        except Exception as e:
            print(f"\n❌ An error occurred during testing: {e}")

🚀 Meeting Analysis Tool (V2 - Markdown Output)

--- Analyzing English Meeting Text (expecting multiple chunks) ---

Analysis Result (Markdown String - English):
### Topic: Efficient AI Model Deployment Using Intel Platforms  

The episode explores the collaboration between Hugging Face and Intel to optimize AI model deployment on Intel CPUs and AI accelerators. Intel's fourth-generation Xeon Scalable processors with Advanced Matrix Extensions (AMX) accelerate matrix multiplications and support data types like BFloat16 and Integer 8. The third-generation Gaudi AI accelerator (Gaudi 3) features 128 GB of RAM and enhanced performance in FP8 and BFloat16 formats. Optimum Intel and Optimum Habana libraries enable seamless integration of these technologies into existing workflows. A live demo showcased Text Generation Inference (TGI) on Gaudi 3 for deploying large language models like Llama 3.17B efficiently. The partnership focuses on providing scalable and efficient solutions for enterpris